In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 50)

target_folder_path = '/Volumes/Database/Original_data'
%cd "{target_folder_path}"

/Volumes/Database/Original_data


### Get ohlcv_df

In [2]:
price_data = pd.read_table(rf"{target_folder_path}/股價資料庫/未調整股價.txt", encoding='Big5-HKSCS', low_memory=False)
price_data

,證券代碼,簡稱,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),報酬率％
0,1101,台泥,19800104,24.9000,25.5,24.9,25.4,259,2.0080
1,1101,台泥,19800105,25.6000,25.8,25.4,25.7,270,1.1811
2,1101,台泥,19800107,25.7000,26.4,25.7,26.0,312,1.1673
3,1101,台泥,19800108,26.0000,26.3,25.8,25.8,224,-0.7692
4,1101,台泥,19800109,25.9000,26.1,25.7,26.0,159,0.7752
...,...,...,...,...,...,...,...,...,...
9338340,8934,衡平,20201111,34.0000,34.0,34.0,34.0,7,3.0303
9338341,8934,衡平,20201112,34.5000,34.5,34.1,34.1,62,0.2941
9338342,8934,衡平,20201113,34.5000,36.5,34.3,36.5,94,7.0381
9338343,8934,衡平,20201116,36.0000,36.0,36.0,36.0,30,-1.3699


In [4]:
columns_list = price_data.columns.tolist()

### make all_columns to str
for column in columns_list:
    price_data[column] = price_data[column].astype(str).str.strip()

ohlcv_list = price_data.columns.tolist()[3:]
ohlcv_list

['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '報酬率％']

In [5]:
price_data['年月日'] = pd.to_datetime(price_data['年月日'], format='%Y%m%d')
# data_full['漲跌停'] =  data_full['漲跌停'].astype(str).str.strip()
# data_full['證券代碼'] = data_full['證券代碼'].str.extract(r'^(\d+)')
string_list = ['證券代碼']
for column_name in string_list:
    price_data[column_name] = price_data[column_name]

numeric_list = ohlcv_list
for column_name in numeric_list:
    price_data[column_name] = pd.to_numeric(price_data[column_name], errors='coerce')


price_data.rename(columns ={'證券代碼':'symbol', 
                            '年月日':'datetime', 
                            '開盤價(元)': 'open',
                            '最高價(元)': 'high',
                            '最低價(元)': 'low', 
                            '收盤價(元)': 'close',
                            '成交量(千股)': 'volume',
                            '報酬率％': 'return',
                           '成交量(千股)':'volume'}, inplace=True)
price_data

ValueError: time data "1980-01-04" doesn't match format "%Y%m%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [6]:
unstack_ohlcv = price_data.reset_index().set_index(['datetime','symbol'])['開盤價(元)'].unstack()
unstack_ohlcv

KeyError: "None of ['datetime', 'symbol'] are in the columns"

In [5]:
ohlcv_list_english = price_data.columns.tolist()[3:]
ohlcv_list_english

['open', 'high', 'low', 'close', 'volume', 'return']

In [6]:
for column in ohlcv_list_english:
    unstack_ohlcv = price_data.reset_index().set_index(['datetime','symbol'])[column].unstack()
    csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column}.csv')
    unstack_ohlcv.to_csv(csv_file)
